<a href="https://colab.research.google.com/github/welting82/AIclass/blob/main/titanic/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from urllib.request import urlretrieve
url = "https://github.com/welting82/AIclass/raw/main/titanic/test.csv"
urlretrieve(url, "test.csv")
url = "https://github.com/welting82/AIclass/raw/main/titanic/train.csv"
urlretrieve(url, "train.csv")

('train.csv', <http.client.HTTPMessage at 0x7f343bde9c90>)

In [38]:
import pandas as pd
train = pd.read_csv("train.csv", encoding="utf-8")
test = pd.read_csv("test.csv", encoding="utf-8")

In [39]:
datas = pd.concat([train,test],axis=0,ignore_index=True)
datas = datas.drop(["PassengerId","Survived"],axis=1)


In [40]:
#demo
name = "Braund, Mr. Owen Harris"
name.split(",")[-1].split(".")[0].strip()

'Mr'

In [41]:
def nameflow(name):
  return name.split(",")[-1].split(".")[0].strip()
datas["Name"] = datas["Name"].apply(nameflow)
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,Mr,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,Mrs,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,Miss,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,Mrs,female,35.0,1,0,113803,53.1000,C123,S
4,3,Mr,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
1304,3,Mr,male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,1,Dona,female,39.0,0,0,PC 17758,108.9000,C105,C
1306,3,Mr,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,3,Mr,male,NaN,0,0,359309,8.0500,NaN,S


In [42]:
def cabinflow(cabin):
    if pd.isna(cabin):
        return cabin
    else:
        return cabin[0]
datas["Cabin"] = datas["Cabin"].apply(cabinflow)
datas["Cabin"].value_counts()

C    94
B    65
D    46
E    41
A    22
F    21
G     5
T     1
Name: Cabin, dtype: int64

In [43]:
dic = datas["Ticket"].value_counts()
def ticketflow(ticket):
  if pd.isna(ticket):
    return ticket
  else:
    return dic[ticket]
datas["Ticket"] = datas["Ticket"].apply(ticketflow)

In [44]:
s = datas.isna().sum()
s[s != 0].sort_values(ascending=False)

Cabin       1014
Age          263
Embarked       2
Fare           1
dtype: int64

In [45]:
#類別型態補值
most = datas["Embarked"].value_counts().idxmax()
datas["Embarked"] = datas["Embarked"].fillna(most)

In [46]:
#數值型態補值
med = datas.median().drop(["Pclass"])
datas = datas.fillna(med)

In [47]:
s = datas.isna().sum()
s[s != 0].sort_values(ascending=False)

Cabin    1014
dtype: int64

In [48]:
# 把出現率太稀少的刪除
c = datas["Name"].value_counts()
reserved = c[c > 50].index
def nameflow2(mid):
    if mid in reserved:
        return mid
    else:
        return None
datas["Name"] = datas["Name"].apply(nameflow2)

In [49]:
# one-hot
datas = pd.get_dummies(datas)
datas = pd.get_dummies(datas, columns=["Pclass"])
datas

,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,22.0,1,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,38.0,1,0,2,71.2833,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
2,26.0,0,0,1,7.9250,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,35.0,1,0,2,53.1000,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0
4,35.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1305,39.0,0,0,3,108.9000,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
1306,38.5,0,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1307,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [50]:
# get rows .iloc -> [fist row, second row] 
x = datas.iloc[:len(train)]
y = train["Survived"]
x_predict = datas.iloc[len(train):]

In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
params = {
    "n_estimators":range(11,150,2),
    "max_depth":range(4,9),

}
clf = RandomForestClassifier()
search = GridSearchCV(clf, params, cv=10, n_jobs=-1)
#search.fit(x,y)

GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': range(4, 9),
                         'n_estimators': range(11, 150)})

In [52]:
print(search.best_score_)
print(search.best_params_)

0.8451310861423222
{'max_depth': 7, 'n_estimators': 19}


In [53]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
clf = DecisionTreeClassifier(max_depth=4)
scores = cross_val_score(clf, x, y, cv=10, n_jobs=-1)
print(scores)
print(np.average(scores))

[0.83333333 0.87640449 0.76404494 0.85393258 0.85393258 0.80898876
 0.82022472 0.7752809  0.88764045 0.85393258]
0.8327715355805244
